<a href="https://colab.research.google.com/github/AnnJer/Practice_in_Data_Preprocessing/blob/master/image_augmenting_cifar10_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np

from keras import backend as K
import keras, tensorflow
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt



Using TensorFlow backend.


In [0]:
EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = RMSprop()
VALIDATION_SPLIT = 0.2
NB_CLASSES = 10
INPUT_SHAPE = (32, 32, 3)

NUM_TO_AUGMENT=5

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 43s 0us/step


In [0]:
from google.colab import files


datagen = ImageDataGenerator(rotation_range = 40, 
                             width_shift_range = 0.2, 
                             height_shift_range = 0.2,
                             zoom_range = 0.2, 
                             horizontal_flip = True, 
                             fill_mode = 'nearest')
xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0

    x = X_train[i] # (28, 28)
    x = x.reshape((1,) + x.shape)  # (1, 32, 32)
#     x = x.reshape((1,) + x.shape)  # (1, 1, 32, 32)
 
    for x_aug in datagen.flow(x, batch_size=1,
    	save_to_dir=None, save_prefix='mnist', save_format='jpeg'):
      if num_aug >= NUM_TO_AUGMENT:
        break
      xtas.append(x_aug[0])
      num_aug += 1

In [0]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

y_train = np_utils.to_categorical(y_train, NB_CLASSES)
y_test = np_utils.to_categorical(y_test, NB_CLASSES)

In [0]:
class SophisticatedModel:
  @staticmethod
  def build(input_shape, classes):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding = 'same', input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense (NB_CLASSES))
    model.add(Activation('softmax'))
    return model

In [0]:
model1 = SophisticatedModel.build(input_shape = X_train.shape[1:], classes = NB_CLASSES)

datagen.fit(X_train)

model1.compile(loss = 'categorical_crossentropy', optimizer = OPTIMIZER, 
             metrics = ['accuracy'])
history = model1.fit_generator(datagen.flow(X_train, y_train, batch_size = BATCH_SIZE), 
                 samples_per_epoch= X_train.shape[0],  epochs = EPOCH, verbose = VERBOSE)
score = model1.evaluate(X_test, y_test, verbose = VERBOSE,batch_size = BATCH_SIZE)
model1.save_weights("model1.h5")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=20, verbose=1, steps_per_epoch=390)`
  


Epoch 1/20
390/390 [==============================] - 35s 90ms/step - loss: 1.9539 - acc: 0.2854
Epoch 2/20
390/390 [==============================] - 31s 80ms/step - loss: 1.6667 - acc: 0.3966
Epoch 3/20
390/390 [==============================] - 31s 80ms/step - loss: 1.5381 - acc: 0.4457
Epoch 4/20
390/390 [==============================] - 31s 80ms/step - loss: 1.4525 - acc: 0.4761
Epoch 5/20
390/390 [==============================] - 31s 80ms/step - loss: 1.3826 - acc: 0.5051
Epoch 6/20
390/390 [==============================] - 31s 80ms/step - loss: 1.3404 - acc: 0.5185
Epoch 7/20
390/390 [==============================] - 31s 80ms/step - loss: 1.2962 - acc: 0.5388
Epoch 8/20
390/390 [==============================] - 31s 80ms/step - loss: 1.2624 - acc: 0.5510
Epoch 9/20
390/390 [==============================] - 31s 79ms/step - loss: 1.2519 - acc: 0.5575
Epoch 10/20
390/390 [==============================] - 31s 80ms/step - loss: 1.2212 - acc: 0.5655
Epoch 11/20
390/390 [========

In [0]:
model_json = model1.to_json()
open('cifar10-_arhitecture.json', 'w').write(model_json)
model1.save_weights('cifar10_weihts.h5', overwrite = True)